In [2]:
## Imports
import pandas as pd
import numpy as np
import networkx as nx
import pickle

In [3]:
## Read data
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
## Genre similarity function
def genre_simi(a, b):
        overlap = 0
        for gens in a:
            if gens in b:
                overlap += 1
        all_included = len(np.unique(np.array(a + b)))
        sim = overlap / all_included

        return sim

In [ ]:
## Movie relation graph
mov_graph = nx.Graph(name='Movie Similarity Graph')
for i in range(len(movies)):
    gen = movies.iloc[i, 2]
    gen_type = gen.split('|')
    for j in range(i + 1, len(movies)):
        genn = movies.iloc[j, 2]
        genn_type = genn.split('|')
        sim = genre_simi(gen_type, genn_type)
        if sim != 0.0:
            mov_graph.add_edge(movies.iloc[i, 0], movies.iloc[j, 0], genre_similarity=sim)

print(mov_graph)

In [28]:
## Save the graph to local
pickle.dump(mov_graph, open('Movie_Similarity_Graph.pickle', 'wb'))

In [7]:
movi_graph = pickle.load(open('Movie_Similarity_Graph.pickle', 'rb'))

In [ ]:
## Weighted PageRank
r = nx.pagerank(movi_graph, alpha=0.85, personalization=person)

In [6]:
## Similarity calculation function
def eud(base, x, y):
    x_r = base.loc[x].values
    y_r = base.loc[y].values
    eudist = 1 / (1 + np.linalg.norm(x_r - y_r))
    return eudist

In [7]:
## Construct the rating map
user = ratings['userId'].unique()
r_map_nan = pd.DataFrame(columns=movies['movieId'].to_list(), index=user)
for i in range(len(ratings)):
    ind = ratings['userId'][i]
    col = ratings['movieId'][i]
    r_map_nan.at[ind, col] = ratings['rating'][i]

r_map = r_map_nan.fillna(0)
r_map.head().iloc[:, :15]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
test = r_map.iloc[:, :1000]
test_nan = r_map_nan.iloc[:, :1000]

In [20]:
## Construct similarity graph
def user_sim(r_matrix):
    W = pd.DataFrame(columns=user, index=user)
    for u in user:
        for v in user:
            if u == v:
                W.at[u, v] = 0
            else:
                W.at[u, v] = round(eud(r_matrix, u, v), 3)
    W.to_csv('user_similarity_matrix', sep=',')
    return W

user_sim(test)

,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
1,0,0.023,0.023,0.02,0.022,0.014,0.024,0.022,0.023,0.023,...,0.023,0.019,0.014,0.019,0.024,0.016,0.022,0.022,0.022,0.02
2,0.023,0,0.111,0.025,0.038,0.016,0.042,0.038,0.068,0.091,...,0.05,0.024,0.014,0.027,0.04,0.016,0.029,0.022,0.047,0.021
3,0.023,0.111,0,0.025,0.038,0.015,0.041,0.037,0.066,0.085,...,0.047,0.024,0.014,0.027,0.041,0.016,0.029,0.022,0.045,0.021
4,0.02,0.025,0.025,0,0.023,0.014,0.024,0.022,0.024,0.025,...,0.025,0.019,0.015,0.02,0.023,0.016,0.022,0.019,0.023,0.019
5,0.022,0.038,0.038,0.023,0,0.016,0.032,0.036,0.034,0.038,...,0.033,0.026,0.014,0.026,0.034,0.016,0.027,0.022,0.035,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.016,0.016,0.016,0.016,0.016,0.013,0.017,0.016,0.016,0.016,...,0.016,0.015,0.015,0.015,0.016,0,0.016,0.016,0.016,0.016
607,0.022,0.029,0.029,0.022,0.027,0.015,0.029,0.027,0.027,0.028,...,0.027,0.022,0.014,0.023,0.027,0.016,0,0.023,0.028,0.022
608,0.022,0.022,0.022,0.019,0.022,0.016,0.023,0.024,0.022,0.022,...,0.022,0.021,0.014,0.021,0.023,0.016,0.023,0,0.023,0.022
609,0.022,0.047,0.045,0.023,0.035,0.016,0.035,0.039,0.04,0.045,...,0.036,0.025,0.014,0.027,0.035,0.016,0.028,0.023,0,0.021


In [9]:
## Weight normalization
def norm_W(w):
    for id in range(len(w)):
        summ = sum(w.iloc[id, :])
        for z in range(len(w)):
            ori = w.iloc[id, :].iat[z]
            w.iloc[id, :].iat[z] = ori/summ
    w.to_csv('user_similarity_matrix_normalized', sep=',')
    return w

W = pd.read_csv('user_similarity_matrix', sep=',')
norm_w = norm_W(W)
norm_w.iloc[:, :10]

,1,2,3,4,5,6,7,8,9,10
1,0.000000,0.001711,0.001711,0.001488,0.001637,0.001042,0.001786,0.001637,0.001711,0.001711
2,0.000718,0.000000,0.003463,0.000780,0.001186,0.000499,0.001310,0.001186,0.002122,0.002839
3,0.000761,0.003674,0.000000,0.000828,0.001258,0.000497,0.001357,0.001225,0.002185,0.002814
4,0.001426,0.001783,0.001783,0.000000,0.001640,0.000998,0.001711,0.001569,0.001711,0.001783
5,0.001141,0.001970,0.001970,0.001193,0.000000,0.000830,0.001659,0.001867,0.001763,0.001970
...,...,...,...,...,...,...,...,...,...,...
606,0.001634,0.001634,0.001634,0.001634,0.001634,0.001327,0.001736,0.001634,0.001634,0.001634
607,0.001364,0.001798,0.001798,0.001364,0.001674,0.000930,0.001798,0.001674,0.001674,0.001736
608,0.001627,0.001627,0.001627,0.001405,0.001627,0.001183,0.001701,0.001775,0.001627,0.001627
609,0.001015,0.002169,0.002077,0.001061,0.001615,0.000738,0.001615,0.001800,0.001846,0.002077


In [10]:
## NaN detection function
def de_nan(base, rx, row):
    ori = base.iloc[rx].index.to_list()
    cle = row.index.to_list()
    for dex in cle:
        ori.remove(dex)
    return ori

In [11]:
## Reconstruction function
def recons(new_data, template):
    temp = template
    for u in range(len(new_data)):
        for i in range(len(new_data[u][0])):
            temp.at[u+1, new_data[u][0][i]] = new_data[u][1][i]
    return temp

In [12]:
## Nearest-neighbor collaborative filtering
def NNCF(rates, user_sim_m):
    new_map = rates  # Copy the unfinished rating map
    for k in range(len(rates)):
        row = rates.iloc[k].dropna()
        simi = user_sim_m.iloc[k].sort_values(ascending=False).index.to_list()  # Sorted similar users
        if len(row) == 0:
            new_map.iloc[k, :] = 2.8 * np.ones(len(rates.iloc[k]))
        else:
            avg_x = sum(row) / len(row)
            unk = de_nan(rates, k, row)  # Unrated movies

            for va in unk:
                nei = 0
                nbs = []
                nbs_sum = 0

                while len(nbs) < 10 and nei < len(simi):
                    if np.isnan(rates.loc[int(simi[nei]), va]):
                        pass
                    else:
                        nbs.append((simi[nei], rates.loc[int(simi[nei]), va]))
                        nbs_sum += rates.loc[int(simi[nei]), va]
                    nei += 1
                temp = 0

                for b in range(len(nbs)):
                    neighbor = rates.loc[int(nbs[b][0])].dropna()
                    cur_avg = sum(neighbor) / len(neighbor)
                    temp += (nbs[b][1] - cur_avg) * user_sim_m.loc[k+1, nbs[b][0]] / nbs_sum
                new_map.loc[k+1, va] = round(avg_x + temp, 1)
            print('\r', k + 1, end='', flush=True)

    new_map.to_csv('new_map', sep=',')
    return new_map

check = NNCF(test_nan, W)
check.iloc[:, :15]

C:\Users\lyx\AppData\Local\Temp\ipykernel_19036\1803201131.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map.loc[k+1, va] = round(avg_x + temp, 1)


 162

C:\Users\lyx\AppData\Local\Temp\ipykernel_19036\1803201131.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_map.iloc[k, :] = 2.8 * np.ones(len(rates.iloc[k]))


 610

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
1,4.0,4.4,4.0,4.4,4.4,4.0,4.4,4.4,4.4,4.4,4.4,4.4,4.4,4.4,4.4
2,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5
3,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1,1.1
4,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7,3.7
5,4.0,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,3.9,3.9,3.9,3.9,3.9,2.5,3.9,3.9,3.9,2.5,3.9,3.9,3.9,3.5
607,4.0,3.9,3.9,3.9,3.9,3.9,3.9,3.9,3.9,3.9,3.0,3.9,3.9,3.9,3.9
608,2.5,2.0,2.0,2.8,2.8,2.8,2.8,2.8,2.8,4.0,2.8,2.8,2.8,2.8,2.8
609,3.0,3.3,3.3,3.3,3.3,3.3,3.3,3.3,3.3,4.0,3.3,3.3,3.3,3.3,3.3
